In [4]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


# Utility to check if a finger is up
def is_finger_up(landmarks, tip_id, mcp_id, palm_facing_camera):
    """
    Returns True if the finger is extended.
    Uses y-coordinates when palm faces camera.
    Flips logic when back of hand faces camera.
    """
    if palm_facing_camera:
        return landmarks[tip_id].y < landmarks[mcp_id].y
    else:
        return landmarks[tip_id].y > landmarks[mcp_id].y


def is_thumb_up(landmarks, hand_label):
    """
    Thumb points outward differently for Left vs Right hand.
    """
    if hand_label == "Right":
        return landmarks[4].x < landmarks[3].x
    else:  # Left hand
        return landmarks[4].x > landmarks[3].x


def detect_gesture(landmarks, hand_label):
    """
    Classifies gesture based on landmark positions.
    """

    # Estimate palm orientation: check z of fingertips vs MCP
    # If fingertips closer to camera (smaller z) -> palm facing camera
    palm_facing = landmarks[8].z < landmarks[5].z  # index tip vs base

    # Finger checks
    index_up = is_finger_up(landmarks, 8, 5, palm_facing)
    middle_up = is_finger_up(landmarks, 12, 9, palm_facing)
    ring_up = is_finger_up(landmarks, 16, 13, palm_facing)
    pinky_up = is_finger_up(landmarks, 20, 17, palm_facing)
    thumb_up = is_thumb_up(landmarks, hand_label)

    # Gesture rules
    if index_up and middle_up and ring_up and pinky_up and thumb_up:
        return "Open Palm"
    elif not index_up and not middle_up and not ring_up and not pinky_up and not thumb_up:
        return "Fist"
    elif index_up and middle_up and not ring_up and not pinky_up:
        return "Peace"
    elif thumb_up and not index_up and not middle_up and not ring_up and not pinky_up:
        return "Thumbs Up"
    else:
        return "Unknown"


# Start webcam
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip and convert to RGB
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)

        # Back to BGR for display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        gesture_name = "No Hand"

        if results.multi_hand_landmarks:
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                hand_label = results.multi_handedness[idx].classification[0].label  # 'Left' or 'Right'
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )
                gesture_name = detect_gesture(hand_landmarks.landmark, hand_label)

        # Show gesture name
        cv2.putText(image, gesture_name, (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Hand Gesture Recognition", image)

        if cv2.waitKey(5) & 0xFF == 27:  # ESC to quit
            break

cap.release()
cv2.destroyAllWindows()



I0000 00:00:1756651630.835504   11589 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1756651630.910067   12460 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.163.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
W0000 00:00:1756651630.917570   12447 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756651630.925435   12444 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
